In [1]:
import numpy as np
import tensorflow as tf
import os
import glob

In [2]:
dogs = ['/home/nishant/Keras/train1/' + filename for filename in os.listdir('/home/nishant/Keras/train1') if filename.startswith("dog")]
cats = ['/home/nishant/Keras/train1/'+filename for filename in os.listdir('/home/nishant/Keras/train1') if filename.startswith("cat")]


In [3]:
filenames =  cats + dogs


In [4]:
print len(filenames)

25000


In [5]:
#creation of labels for the data
labels = []
for x in cats:
    labels.append(0)
    
for x in dogs:
    labels.append(1)

In [6]:
print len(labels)

25000


In [7]:
#creating the dataset from the image paths and labels
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

dataset = dataset.shuffle(len(filenames))

In [8]:
print dataset

<ShuffleDataset shapes: ((), ()), types: (tf.string, tf.int32)>


In [9]:
#function for parsing the image
def parse_function(filename, label):
    image_string = tf.read_file(filename)

    image = tf.image.decode_jpeg(image_string, channels=3)

    # Conversion to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_images(image, [64, 64])
    return image, label

#function for augmentation
def train_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)

    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label


In [10]:
batch_size = 128
dataset = dataset.map(parse_function, num_parallel_calls=4)
dataset1 = dataset.map(train_preprocess, num_parallel_calls=4)
dataset = dataset.concatenate(dataset1)
dataset = dataset.batch(batch_size)



In [11]:
iter = dataset.make_one_shot_iterator()
x, y = iter.get_next()
print y
print x



Tensor("IteratorGetNext:1", shape=(?,), dtype=int32)
Tensor("IteratorGetNext:0", shape=(?, 64, 64, 3), dtype=float32)


In [12]:
def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev= 0.1))

def init_bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding="SAME")

def max_pool(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1], padding ="SAME")

def convolution_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [13]:
#Defining the first block of the vgg model
x = tf.reshape(x,shape = [-1,64,64,3])
convo_1 = convolution_layer(x,shape=[3,3,3,64])
convo_2 = convolution_layer(convo_1,shape=[3,3,64,64])
pool_1 = max_pool(convo_2)
fc = tf.reshape(pool_1,shape = [-1,32*32*64])
fc_1 = normal_full_layer(fc,1000)
fc_2 = normal_full_layer(fc_1,2)

logits = tf.nn.softmax(fc_2)

labels = y
labels = tf.cast(labels, tf.int64)

# Defining the prediction as the argmax of the scores
predictions = tf.argmax(logits, 1)

# Defining the loss
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
matches = tf.equal(predictions, labels)
accuracy = tf.reduce_mean(tf.cast(matches,tf.float32))
optimizer = tf.train.AdamOptimizer(0.01)

In [14]:

# Creating the training operation
train_op = optimizer.minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(190):
        _, loss_val = sess.run([train_op, loss])
        print i
        print('Loss:')
        print loss_val
        print('Accuracy:')
        print sess.run(accuracy)



0
Loss:
0.7286252
Accuracy:
0.5703125
1
Loss:
0.85232425
Accuracy:
0.53125
2
Loss:
0.75857425
Accuracy:
0.5078125
3
Loss:
0.80544925
Accuracy:
0.5078125
4
Loss:
0.79763675
Accuracy:
0.4609375
5
Loss:
0.83669925
Accuracy:
0.46875
6
Loss:
0.80544925
Accuracy:
0.4453125
7
Loss:
0.86013675
Accuracy:
0.515625
8
Loss:
0.86013675
Accuracy:
0.46875
9
Loss:
0.82107425
Accuracy:
0.515625
10
Loss:
0.86013675
Accuracy:
0.4453125
11
Loss:
0.82107425
Accuracy:
0.53125
12
Loss:
0.86013675
Accuracy:
0.4375
13
Loss:
0.80544925
Accuracy:
0.53125
14
Loss:
0.83669925
Accuracy:
0.5078125
15
Loss:
0.75076175
Accuracy:
0.5078125
16
Loss:
0.75857425
Accuracy:
0.609375
17
Loss:
0.84451175
Accuracy:
0.5078125
18
Loss:
0.82107425
Accuracy:
0.6015625
19
Loss:
0.83669925
Accuracy:
0.4921875
20
Loss:
0.83669925
Accuracy:
0.5234375
21
Loss:
0.82888675
Accuracy:
0.484375
22
Loss:
0.75857425
Accuracy:
0.515625
23
Loss:
0.78201175
Accuracy:
0.4296875
24
Loss:
0.79763675
Accuracy:
0.46875
25
Loss:
0.82888675
Accuracy:
0